<a href="https://colab.research.google.com/github/tec03/ParaAssignment01-/blob/main/assignment01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TC2029 Ciencia y Analitica de Datos
Dra. María de la Paz Rico Fernández 

Juan Pablo Bladinieres Martin del Campo A01793474
Gerardo Quiroga                         A00967999

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Point
import qeds
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import recall_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
qeds.themes.mpl_style();

In [2]:
# Transformación completa de los resultados del analisis
path1='./Datos_de_calidad_del_agua_2020/Datos_de_calidad_del_agua_de_sitios_de_monitoreo_de_aguas_superficiales_2020.csv'
path2='./Datos_de_calidad_del_agua_2020/Datos_de_calidad_del_agua_de_sitios_de_monitoreo_de_aguas_subterraneas_2020.csv'
df=pd.read_csv(path1)

df.rename(mapper=df['CLAVE'],axis=0,inplace=True)

df["Coordinates"] = list(zip(df.LONGITUD, df.LATITUD))
df["Coordinates"] = df["Coordinates"].apply(Point)

df.drop(['CLAVE','TIPO','CONTAMINANTES','PERIODO','SITIO','ESTADO','MUNICIPIO','CUENCA','CUERPO DE AGUA','ORGANISMO_DE_CUENCA'],axis=1,inplace=True)

df['SUBTIPO']=df["SUBTIPO"].astype("string")
df['SEMAFORO']=df["SEMAFORO"].astype("string")
df['GRUPO']=df["GRUPO"].astype("string")
df['SUBTIPO']=df["SUBTIPO"].str.upper()
df['SEMAFORO']=df['SEMAFORO'].replace({'Amarillo':'y','Rojo':'r','Verde':'g'})

localizacion=['LATITUD','LONGITUD']
for i in localizacion:
    df[i]=df[i].astype("float")
    
numericas=['DBO_mg/L','DQO_mg/L','SST_mg/L','COLI_FEC_NMP_100mL','E_COLI_NMP_100mL','ENTEROC_NMP_100mL','OD_PORC','OD_PORC_SUP','OD_PORC_MED','OD_PORC_FON','TOX_V_15_UT','TOX_D_48_UT','TOX_D_48_SUP_UT','TOX_D_48_FON_UT','TOX_FIS_SUP_15_UT','TOX_FIS_FON_15_UT']
for i in numericas:
    df[i]=df[i].replace({np.NAN:0})
    df[i]=df[i].replace({'<2':0,'<10':0,'<3':0,'<1':0})
    df[i]=df[i].astype("float")

calidades1=['CALIDAD_DBO','CALIDAD_DQO','CALIDAD_SST','CALIDAD_COLI_FEC','CALIDAD_E_COLI','CALIDAD_ENTEROC','CALIDAD_OD_PORC','CALIDAD_OD_PORC_SUP','CALIDAD_OD_PORC_MED','CALIDAD_OD_PORC_FON']
for i in calidades1:
    df[i]=df[i].replace({np.NAN:'Sin Medida'})
    df[i]=df[i].astype("string")

calidades2=['CALIDAD_TOX_V_15','CALIDAD_TOX_D_48','CALIDAD TOX_D_48_SUP','CALIDAD_TOX_D_48_FON','CALIDAD_TOX_FIS_SUP_15','CALIDAD_TOX_FIS_FON_15']
for i in calidades2:
    df[i]=df[i].replace({np.NAN:'Sin Medida'})
    df[i]=df[i].astype("string")

binarias=['CUMPLE_CON_DBO','CUMPLE_CON_DQO','CUMPLE_CON_SST','CUMPLE_CON_CF','CUMPLE_CON_E_COLI','CUMPLE_CON_ENTEROC','CUMPLE_CON_OD','CUMPLE_CON_TOX']
for i in binarias:
    df[i]=df[i].replace({'ND':0,'NO':0,'SI':1})
    df[i]=df[i].astype("float")


df.describe(include='all')

c:\Python310\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,SUBTIPO,LONGITUD,LATITUD,DBO_mg/L,CALIDAD_DBO,DQO_mg/L,CALIDAD_DQO,SST_mg/L,CALIDAD_SST,COLI_FEC_NMP_100mL,...,CUMPLE_CON_DBO,CUMPLE_CON_DQO,CUMPLE_CON_SST,CUMPLE_CON_CF,CUMPLE_CON_E_COLI,CUMPLE_CON_ENTEROC,CUMPLE_CON_OD,CUMPLE_CON_TOX,GRUPO,Coordinates
count,3479,3493.000000,3493.000000,4141.000000,4141,4141.000000,4141,4141.000000,4141,4.141000e+03,...,3493.000000,3493.000000,3493.000000,3493.000000,3493.000000,3493.000000,3493.000000,3493.000000,3493,4141
unique,23,NaN,NaN,NaN,6,NaN,6,NaN,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3492
top,RIO,NaN,NaN,NaN,Sin Medida,NaN,Sin Medida,NaN,Excelente,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LOTICO,POINT EMPTY
freq,1478,NaN,NaN,NaN,1560,NaN,1560,NaN,1780,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1772,648
mean,NaN,-100.359969,21.046992,9.933834,NaN,38.747509,NaN,83.939839,NaN,5.966389e+04,...,0.663899,0.472373,0.888634,0.271400,0.415975,0.215288,0.846837,0.976524,NaN,NaN
std,NaN,6.122773,3.893696,52.175019,NaN,122.697137,NaN,407.697458,NaN,9.240891e+05,...,0.472441,0.499308,0.314629,0.444746,0.492960,0.411081,0.360197,0.151430,NaN,NaN
min,NaN,-117.124030,14.534910,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
25%,NaN,-103.882310,18.396070,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000e+00,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,NaN,NaN
50%,NaN,-99.795530,20.148980,0.000000,NaN,0.000000,NaN,18.000000,NaN,2.100000e+02,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,NaN,NaN
75%,NaN,-96.860230,22.828930,5.000000,NaN,35.750000,NaN,48.000000,NaN,4.600000e+03,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,NaN,NaN


Se debe tomar la decisión de como manejar los datos con categorias o con datos númericos. Por esta razón se decide usar los categoricos y se divide la base en X y Y para su modelaje los Y son los datos de salida en este caso nuestra variable de SEMAFORO.

In [20]:
X = df.copy()
X.drop(binarias, axis=1,inplace=True)
X.drop(numericas, axis=1,inplace=True)
X.drop(['Coordinates'], axis=1,inplace=True)
y = df['SEMAFORO'].copy()

print(X.shape)
print(y.shape)

(4141, 21)
(4141,)


In [21]:
X_tv,X_test,y_tv,y_test = train_test_split(X,y, test_size=.20, shuffle=True, random_state=1)

print(X_tv.shape)
print(y_tv.shape)
print(X_test.shape)
print(y_test.shape)

(3312, 21)
(3312,)
(829, 21)
(829,)


In [26]:
# Clasificador 
pipe = Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent')),
                        ('onehot',OneHotEncoder()),
                      ])

X_train,y_train,X_val,y_val = train_test_split(X_tv,y_tv, test_size=.20, shuffle=True, random_state=1)


print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)




(2649, 21)
(663, 21)
(2649,)
(663,)


In [28]:
# RANDOM FOREST MODEL
rfc=RandomForestClassifier(max_depth=5)
pipe.fit_transform(X_train,y_train)
rfc.fit(X_train,y_train)
y_hat_rfc = rfc.predict(X_val)
print('RFC RECALL: ',recall_score(y_val,y_hat_rfc))

AttributeError: 'bool' object has no attribute 'transpose'

In [ ]:
# DECISION TREE MODEL
dtc=DecisionTreeClassifier(max_depth=5)
dtc.fit(X_train,y_train)
y_hat_dtc = dtc.predict(X_val)
print('DTC RECALL: ',recall_score(y_val,y_hat_dtc))